In [1]:
"""
Bioreactor ML Starter Code
===========================

Quick-start template for building ML models on bioreactor data.
Designed for speed and practicality, not elegance.

Author: Your Name
Date: Started [Date]
Goal: Predict final titer from early measurements, 8-month timeline
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PART 1: Data Loading & Cleaning (ADAPT TO YOUR DATA FORMAT)
# ============================================================================

class BioreactorDataLoader:
    """
    Load and clean bioreactor data.
    Adapt the load_run() method to your specific data format.
    """
    
    def __init__(self, data_dir: str):
        self.data_dir = Path(data_dir)
        self.runs = {}
        
    def load_run(self, filepath: Path) -> pd.DataFrame:
        """
        Load one bioreactor run. 
        
        ADAPT THIS to your data format:
        - CSV from bioreactor software?
        - Excel sheets?
        - Database export?
        - Custom format?
        """
        # Example for CSV with columns: time, OD600, pH, DO, substrate, etc.
        try:
            df = pd.read_csv(filepath)
            
            # Standardize column names
            df.columns = df.columns.str.lower().str.strip()
            
            # Convert time to hours from start
            if 'time' in df.columns or 'timestamp' in df.columns:
                time_col = 'time' if 'time' in df.columns else 'timestamp'
                df[time_col] = pd.to_datetime(df[time_col])
                df['hours'] = (df[time_col] - df[time_col].min()).dt.total_seconds() / 3600
            
            return df
            
        except Exception as e:
            print(f"Error loading {filepath}: {e}")
            return None
    
    def load_all_runs(self, pattern: str = "*.csv") -> Dict[str, pd.DataFrame]:
        """Load all runs matching pattern"""
        files = list(self.data_dir.glob(pattern))
        print(f"Found {len(files)} files matching '{pattern}'")
        
        for f in files:
            run_id = f.stem
            df = self.load_run(f)
            if df is not None:
                self.runs[run_id] = df
        
        print(f"Successfully loaded {len(self.runs)} runs")
        return self.runs
    
    def get_metadata(self, metadata_file: str = None) -> pd.DataFrame:
        """
        Load metadata (strain info, media, final titer, etc.)
        
        Expected format (CSV):
        run_id, strain, media, final_titer_g_L, harvest_time_h, notes
        """
        if metadata_file is None:
            metadata_file = self.data_dir / "metadata.csv"
        
        if Path(metadata_file).exists():
            return pd.read_csv(metadata_file)
        else:
            print(f"Warning: No metadata file found at {metadata_file}")
            return None


class BioreactorDataCleaner:
    """
    Clean and preprocess bioreactor time-series data.
    """
    
    def __init__(self):
        self.feature_names = []
    
    def align_time(self, df: pd.DataFrame, time_col: str = 'hours') -> pd.DataFrame:
        """Ensure time starts at 0"""
        df = df.copy()
        df[time_col] = df[time_col] - df[time_col].min()
        return df
    
    def interpolate_missing(self, df: pd.DataFrame, method: str = 'linear') -> pd.DataFrame:
        """Handle missing values"""
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        df[numeric_cols] = df[numeric_cols].interpolate(method=method)
        return df
    
    def remove_outliers(self, df: pd.DataFrame, cols: List[str], 
                       n_std: float = 3) -> pd.DataFrame:
        """Remove outliers beyond n standard deviations"""
        df = df.copy()
        for col in cols:
            if col in df.columns:
                mean = df[col].mean()
                std = df[col].std()
                df[col] = df[col].clip(mean - n_std*std, mean + n_std*std)
        return df
    
    def resample_uniform(self, df: pd.DataFrame, interval_hours: float = 0.5) -> pd.DataFrame:
        """Resample to uniform time intervals"""
        df = df.set_index('hours')
        df = df.resample(f'{interval_hours}H').mean().interpolate()
        return df.reset_index()
    
    def clean_run(self, df: pd.DataFrame) -> pd.DataFrame:
        """Full cleaning pipeline"""
        df = self.align_time(df)
        df = self.interpolate_missing(df)
        df = self.remove_outliers(df, ['od600', 'ph', 'do'])
        return df


# ============================================================================
# PART 2: Feature Engineering
# ============================================================================

class BioreactorFeatureExtractor:
    """
    Extract features for ML from time-series data.
    """
    
    def extract_early_features(self, df: pd.DataFrame, 
                              time_window: float = 24.0) -> Dict[str, float]:
        """
        Extract features from early time window.
        
        This is the key: predict final titer from early measurements!
        """
        early_df = df[df['hours'] <= time_window]
        
        features = {}
        
        # OD600 features
        if 'od600' in df.columns:
            features['od_24h'] = early_df['od600'].iloc[-1] if len(early_df) > 0 else np.nan
            features['od_max_24h'] = early_df['od600'].max()
            features['od_mean_24h'] = early_df['od600'].mean()
            
            # Growth rate (exponential phase)
            if len(early_df) > 5:
                od_log = np.log(early_df['od600'].replace(0, np.nan).dropna())
                if len(od_log) > 2:
                    # Linear fit to log(OD) vs time
                    time = early_df.loc[od_log.index, 'hours']
                    growth_rate = np.polyfit(time, od_log, 1)[0]
                    features['growth_rate'] = growth_rate
        
        # pH features
        if 'ph' in df.columns:
            features['ph_24h'] = early_df['ph'].iloc[-1] if len(early_df) > 0 else np.nan
            features['ph_min_24h'] = early_df['ph'].min()
            features['ph_drop_24h'] = early_df['ph'].iloc[0] - early_df['ph'].iloc[-1] if len(early_df) > 1 else 0
        
        # DO features  
        if 'do' in df.columns:
            features['do_24h'] = early_df['do'].iloc[-1] if len(early_df) > 0 else np.nan
            features['do_min_24h'] = early_df['do'].min()
        
        # Substrate features
        if 'substrate' in df.columns:
            features['substrate_24h'] = early_df['substrate'].iloc[-1] if len(early_df) > 0 else np.nan
            features['substrate_consumed_24h'] = early_df['substrate'].iloc[0] - early_df['substrate'].iloc[-1] if len(early_df) > 1 else 0
        
        return features
    
    def extract_all_features(self, runs: Dict[str, pd.DataFrame],
                           metadata: pd.DataFrame = None) -> pd.DataFrame:
        """Extract features from all runs"""
        
        feature_list = []
        
        for run_id, df in runs.items():
            features = self.extract_early_features(df)
            features['run_id'] = run_id
            
            # Add metadata if available
            if metadata is not None and run_id in metadata['run_id'].values:
                meta_row = metadata[metadata['run_id'] == run_id].iloc[0]
                features['final_titer'] = meta_row.get('final_titer_g_l', np.nan)
                features['strain'] = meta_row.get('strain', 'unknown')
                features['media'] = meta_row.get('media', 'unknown')
            
            feature_list.append(features)
        
        feature_df = pd.DataFrame(feature_list)
        print(f"\nExtracted features from {len(feature_df)} runs")
        print(f"Feature columns: {feature_df.columns.tolist()}")
        
        return feature_df


# ============================================================================
# PART 3: Baseline ML Models
# ============================================================================

def build_baseline_models(X_train, y_train, X_test, y_test):
    """
    Build and compare baseline models.
    Start simple before going to neural operators!
    """
    from sklearn.linear_model import LinearRegression, Ridge
    from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
    from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
    
    models = {
        'Linear': LinearRegression(),
        'Ridge': Ridge(alpha=1.0),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Gradient Boost': GradientBoostingRegressor(n_estimators=100, random_state=42)
    }
    
    results = {}
    
    print("\n" + "="*70)
    print("BASELINE MODEL COMPARISON")
    print("="*70)
    
    for name, model in models.items():
        # Train
        model.fit(X_train, y_train)
        
        # Predict
        y_pred = model.predict(X_test)
        
        # Metrics
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mae = mean_absolute_error(y_test, y_pred)
        
        results[name] = {
            'model': model,
            'r2': r2,
            'rmse': rmse,
            'mae': mae,
            'predictions': y_pred
        }
        
        print(f"\n{name}:")
        print(f"  R² Score:  {r2:.3f}")
        print(f"  RMSE:      {rmse:.3f} g/L")
        print(f"  MAE:       {mae:.3f} g/L")
    
    return results


def plot_predictions(y_test, results, save_path='baseline_comparison.png'):
    """Visualize prediction quality"""
    n_models = len(results)
    fig, axes = plt.subplots(1, n_models, figsize=(5*n_models, 4))
    
    if n_models == 1:
        axes = [axes]
    
    for idx, (name, result) in enumerate(results.items()):
        ax = axes[idx]
        y_pred = result['predictions']
        
        # Scatter plot: predicted vs actual
        ax.scatter(y_test, y_pred, alpha=0.6, s=60)
        
        # Perfect prediction line
        min_val = min(y_test.min(), y_pred.min())
        max_val = max(y_test.max(), y_pred.max())
        ax.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect')
        
        ax.set_xlabel('Actual Titer (g/L)', fontsize=11)
        ax.set_ylabel('Predicted Titer (g/L)', fontsize=11)
        ax.set_title(f"{name}\nR² = {result['r2']:.3f}", fontsize=12, fontweight='bold')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"\n✓ Comparison plot saved to {save_path}")


# ============================================================================
# PART 4: Complete Workflow
# ============================================================================

def main_workflow(data_dir: str, metadata_file: str = None):
    """
    End-to-end workflow from raw data to trained models.
    
    This is your starting point. Run this and see what happens!
    """
    
    print("\n" + "="*70)
    print("BIOREACTOR ML PIPELINE")
    print("="*70)
    
    # Step 1: Load data
    print("\n[STEP 1] Loading bioreactor data...")
    loader = BioreactorDataLoader(data_dir)
    runs = loader.load_all_runs(pattern="*.csv")  # Adjust pattern as needed
    metadata = loader.get_metadata(metadata_file)
    
    if len(runs) == 0:
        print("ERROR: No data loaded. Check your data_dir and file format.")
        return
    
    # Step 2: Clean data
    print("\n[STEP 2] Cleaning time-series data...")
    cleaner = BioreactorDataCleaner()
    runs_clean = {run_id: cleaner.clean_run(df) for run_id, df in runs.items()}
    
    # Step 3: Extract features
    print("\n[STEP 3] Extracting features...")
    extractor = BioreactorFeatureExtractor()
    features = extractor.extract_all_features(runs_clean, metadata)
    
    # Check if we have target variable
    if 'final_titer' not in features.columns:
        print("\nWARNING: No 'final_titer' column found in metadata.")
        print("You need final titer measurements to train predictive models!")
        print("Features extracted:", features.columns.tolist())
        return features
    
    # Step 4: Prepare train/test split
    print("\n[STEP 4] Preparing train/test split...")
    
    # Remove rows with missing target
    features = features.dropna(subset=['final_titer'])
    
    # Separate features and target
    feature_cols = [c for c in features.columns 
                   if c not in ['run_id', 'final_titer', 'strain', 'media']]
    
    X = features[feature_cols].fillna(features[feature_cols].mean())
    y = features['final_titer']
    
    # Train/test split (80/20)
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    
    print(f"Training samples: {len(X_train)}")
    print(f"Test samples: {len(X_test)}")
    print(f"Features: {feature_cols}")
    
    # Step 5: Build baseline models
    print("\n[STEP 5] Training baseline models...")
    results = build_baseline_models(X_train, y_train, X_test, y_test)
    
    # Step 6: Visualize
    print("\n[STEP 6] Creating visualizations...")
    plot_predictions(y_test.values, results)
    
    # Step 7: Feature importance
    print("\n[STEP 7] Analyzing feature importance...")
    best_model = results['Random Forest']['model']
    importances = pd.DataFrame({
        'feature': feature_cols,
        'importance': best_model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    print("\nTop 5 Most Important Features:")
    print(importances.head())
    
    print("\n" + "="*70)
    print("WORKFLOW COMPLETE!")
    print("="*70)
    print("\nNext steps:")
    print("1. Check baseline_comparison.png to see model performance")
    print("2. If results are good, move to neural operators + transfer learning")
    print("3. If results are poor, check data quality and feature engineering")
    
    return features, results


# ============================================================================
# PART 5: Quick Start Guide
# ============================================================================

def quick_start_guide():
    """
    Print usage instructions.
    """
    print("""
╔══════════════════════════════════════════════════════════════════════╗
║                    BIOREACTOR ML QUICK START                          ║
╚══════════════════════════════════════════════════════════════════════╝

STEP 1: ORGANIZE YOUR DATA
--------------------------
Create this folder structure:

    bioreactor_data/
    ├── run_001.csv
    ├── run_002.csv
    ├── ...
    └── metadata.csv

Each run CSV should have columns like:
    time, od600, ph, do, substrate, temperature

metadata.csv should have:
    run_id, strain, media, final_titer_g_l, harvest_time_h

STEP 2: ADAPT THE CODE
-----------------------
Edit the load_run() method in BioreactorDataLoader to match your format.

STEP 3: RUN THE WORKFLOW
-------------------------
    from bioreactor_ml_starter import main_workflow
    
    features, results = main_workflow(
        data_dir='./bioreactor_data',
        metadata_file='./bioreactor_data/metadata.csv'
    )

STEP 4: INTERPRET RESULTS
--------------------------
Check the output:
    - R² score > 0.7: Good! Move to neural operators
    - R² score 0.4-0.7: Okay, but need more data or better features
    - R² score < 0.4: Check data quality, may need transfer learning

STEP 5: ITERATE
---------------
    - Add more features if needed
    - Try different time windows (12h, 48h instead of 24h)
    - Include strain/media as categorical features
    - Move to neural operators with mechanistic priors

TROUBLESHOOTING
---------------
Q: "No data loaded"
A: Check file paths and format in load_run()

Q: "Not enough samples"
A: Need at least 20 runs, preferably 50+

Q: "Poor predictions"
A: Check if features are informative, may need transfer learning

Q: "Missing final_titer"
A: Need experimental endpoint measurements in metadata

═══════════════════════════════════════════════════════════════════════
                     Ready to start? Let's go! 🚀
═══════════════════════════════════════════════════════════════════════
    """)


# ============================================================================
# RUN IT
# ============================================================================

if __name__ == "__main__":
    # Print guide
    quick_start_guide()
    
    # Example usage (uncomment and modify paths):
    # features, results = main_workflow(
    #     data_dir='./your_bioreactor_data',
    #     metadata_file='./your_bioreactor_data/metadata.csv'
    # )


╔══════════════════════════════════════════════════════════════════════╗
║                    BIOREACTOR ML QUICK START                          ║
╚══════════════════════════════════════════════════════════════════════╝

STEP 1: ORGANIZE YOUR DATA
--------------------------
Create this folder structure:

    bioreactor_data/
    ├── run_001.csv
    ├── run_002.csv
    ├── ...
    └── metadata.csv

Each run CSV should have columns like:
    time, od600, ph, do, substrate, temperature

metadata.csv should have:
    run_id, strain, media, final_titer_g_l, harvest_time_h

STEP 2: ADAPT THE CODE
-----------------------
Edit the load_run() method in BioreactorDataLoader to match your format.

STEP 3: RUN THE WORKFLOW
-------------------------
    from bioreactor_ml_starter import main_workflow

    features, results = main_workflow(
        data_dir='./bioreactor_data',
        metadata_file='./bioreactor_data/metadata.csv'
    )

STEP 4: INTERPRET RESULTS
--------------------------
Check t